In [62]:
import pandas as pd
import numpy as np
import cufflinks
from tabulate import tabulate
from collections import Counter

In [22]:
df = pd.read_csv('Baseball_original.csv')
df[:3]

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,NaN,0.83,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,NaN,0.82,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,NaN,0.81,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1


In [23]:
print(len(df.columns))
df.describe()

48


/usr/local/lib/python3.4/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning:

Invalid value encountered in percentile



,yearID,Rank,G,Ghome,W,L,R,AB,H,2B,...,HA,HRA,BBA,SOA,E,DP,FP,attendance,BPF,PPF
count,2805.000000,2805.000000,2805.00000,2406.000000,2805.000000,2805.000000,2805.000000,2805.000000,2805.000000,2805.000000,...,2805.000000,2805.000000,2805.000000,2805.000000,2805.000000,2488.000000,2805.000000,2.526000e+03,2805.000000,2805.000000
mean,1955.036720,4.107308,150.34795,78.465919,74.749020,74.749020,681.945811,5142.492335,1346.273440,227.624955,...,1346.083779,101.136542,474.010695,731.229234,186.337255,140.186495,0.961519,1.344346e+06,100.199643,100.225668
std,41.519083,2.323414,23.22725,4.698684,17.640402,17.378079,135.738244,750.551691,219.891603,58.692602,...,219.521064,58.245002,131.890032,296.409881,107.657444,29.322764,0.030224,9.469316e+05,4.882215,4.814985
min,1871.000000,1.000000,6.00000,44.000000,0.000000,4.000000,24.000000,211.000000,33.000000,3.000000,...,49.000000,0.000000,0.000000,0.000000,47.000000,18.000000,0.760000,6.088000e+03,60.000000,60.000000
25%,1919.000000,2.000000,153.00000,NaN,66.000000,65.000000,613.000000,5127.000000,1299.000000,193.000000,...,1288.000000,46.000000,427.000000,501.000000,116.000000,NaN,0.960000,NaN,97.000000,97.000000
50%,1963.000000,4.000000,157.00000,NaN,77.000000,76.000000,690.000000,5389.000000,1393.000000,231.000000,...,1392.000000,109.000000,494.000000,735.000000,145.000000,NaN,0.970000,NaN,100.000000,100.000000
75%,1992.000000,6.000000,162.00000,NaN,87.000000,87.000000,763.000000,5517.000000,1467.000000,270.000000,...,1470.000000,148.000000,555.000000,965.000000,217.000000,NaN,0.980000,NaN,103.000000,103.000000
max,2015.000000,13.000000,165.00000,84.000000,116.000000,134.000000,1220.000000,5781.000000,1783.000000,376.000000,...,1993.000000,241.000000,827.000000,1450.000000,639.000000,217.000000,0.991000,4.483350e+06,129.000000,141.000000


In [24]:
df_clean = df.dropna(axis='columns')
print('dropped columns: ')
print([f for f in df if f not in df_clean.columns and "id" not in f.lower()])
print(len(df_clean.columns))

dropped columns: 
['Ghome', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'SO', 'SB', 'CS', 'HBP', 'SF', 'DP', 'park', 'attendance']
33


In [25]:
df.isnull().sum().iplot(kind='bar')

lgID    League: use most common

divID   Team's division: use own encoding

GHome   Games played at home: use int mean

DivWin  Division Winner (Y or N): remove

WCWin   Wild Card Winner (Y or N): remove

LgWin   League Champion(Y or N): use most common ('N')

WSWin   World Series Winner (Y or N): use most common ('N')

SO  Strikeouts by batters: mean

SB  Stolen bases: mean

CS  Caught stealing: median

HBP Batters hit by pitch

SF  Sacrifice flies

DP  Double Plays: mean of 50 above and below (earlier years have lower DP)

park Placename: remove

attendance  Home attendance total: use mean. should be lower for earlier years, but hard to model. maybe mean of years till 1920?


Remove features with many missing values and impute missing values for rest

In [32]:
df_clean = df.loc[:,df.isnull().sum()<0.5*len(df)]
df_clean.isnull().sum().iplot(kind='bar')

In [65]:
cols_to_impute = df_clean.loc[:,df_clean.isnull().sum()>0].columns

print(df_clean[cols_to_impute][:3])
print(df_clean[cols_to_impute][-3:])
print('--')
print(tabulate(df_clean[cols_to_impute].describe(), headers='keys'))

  lgID  Ghome LgWin WSWin    SO    SB  CS  DP                          park  \
0  NaN    NaN     N   NaN  19.0  73.0 NaN NaN           South End Grounds I   
1  NaN    NaN     N   NaN  22.0  69.0 NaN NaN       Union Base-Ball Grounds   
2  NaN    NaN     N   NaN  25.0  18.0 NaN NaN  National Association Grounds   

   attendance  
0         NaN  
1         NaN  
2         NaN  
     lgID  Ghome LgWin WSWin      SO     SB    CS     DP         park  \
2802   NL   81.0     N     N  1312.0  132.0  44.0  146.0  Chase Field   
2803   NL   81.0     N     N  1327.0   82.0  29.0  138.0   Petco Park   
2804   NL   81.0     N     N  1283.0   97.0  43.0  171.0  Coors Field   

      attendance  
2802   2080145.0  
2803   2459742.0  
2804   2506789.0  
--
            Ghome        SO         SB         CS         DP        attendance
-----  ----------  --------  ---------  ---------  ---------  ----------------
count  2406        2685      2661       1946       2488         2526
mean     78.4659    

/usr/local/lib/python3.4/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning:

Invalid value encountered in percentile



In [54]:
print('median gHome:',)
np.median(df_clean.dropna()[df_clean['yearID']>1900]['Ghome'])

median gHome:


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



81.0

In [59]:
set(df_clean['LgWin'])
Counter(df_clean['LgWin'])

Counter({'N': 2505, nan: 28, 'Y': 272})

In [61]:
set(df_clean['WSWin'])
Counter(df_clean['WSWin'])

Counter({nan: 357, 'N': 2332, 'Y': 116})